# Nintendo Tweets Modeling

We've arrived at the modeling phase of the project. As a recap, our data is imbalanced with positively labeled tweets being the large majority and negative tweets being the minority. As such, it is important that our models are able to accurately detect negative tweets, even if it comes at the expense of accurately detecting positive tweets. The primary performance metric we will be looking at is recall, as it tells us how many negative tweets the model is able to correctly label.

Because this is a binary classification problem, we will be building a variety of classification models suited for this kind of problem. I will be working with LogisticRegression, RandomForestClassifier, XGBClassifier, and SVC.

In [1]:
# imports

import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import datetime
import pprint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [162]:
# reading in files

path = "/Users/jasonzhou/Documents/GitHub/NintendoTweets/Documents/Capstone3"
os.chdir(path)

smashtraining = pd.read_csv('smashtraining.csv')
firetraining = pd.read_csv('firetraining.csv')
partytraining = pd.read_csv('partytraining.csv')

smashsamples = pd.read_csv('smashsamples.csv')
firesamples = pd.read_csv('firesamples.csv')
partysamples = pd.read_csv('partysamples.csv')

In [3]:
# function that creates count vectorizers of corpus, and returns counts array, feature names, 
# and the vectorizer itself

def makeCountVec(df):
    vectorizer = CountVectorizer(min_df=0)
    vectorizer.fit(df['cleanedtext'])
    array = vectorizer.transform(df['cleanedtext'])
    array = array.toarray()
    features = vectorizer.get_feature_names()
    return array, features, vectorizer

In [4]:
smashX, smashfeatures, smashvectorizer = makeCountVec(smashtraining)
smashy = smashtraining['label']

fireX, firefeatures, firevectorizer, = makeCountVec(firetraining)
firey = firetraining['label']

partyX, partyfeatures, partyvectorizer = makeCountVec(partytraining)
partyy = partytraining['label']

Here what we need to do is remove the rows corresponding to the manually labeled samples we will use to ultimately validate the models on the accuracy of TextBlob. Because of our work during the preprocessing phase of the project, I just need to drop the last n rows of each X matrix, with n corresponding to the length of each sample.

The reason we do this after the vectorizing instead of before is so that the vectorizers are able to fit to every token in the data. 

In [5]:
smashX = smashX[:-len(smashsamples),]
fireX = fireX[:-len(firesamples), :]
partyX = partyX[:-len(partysamples), :]

In [6]:
# Need to do the same for the y vectors too

smashy = smashy[:-len(smashsamples)]
firey = firey[:-len(firesamples)]
partyy = partyy[:-len(partysamples)]

For reference, each game will be mapped to a number, in the interest of variable name lengths:

- Smash Bros. Ultimate: 1

- Fire Emblem: Three Houses: 2

- Super Mario Party: 3

In [7]:
from sklearn.model_selection import train_test_split

Xtr1, Xte1, ytr1, yte1 = train_test_split(smashX, smashy, test_size=0.3, random_state=1, stratify = smashy)
Xtr2, Xte2, ytr2, yte2 = train_test_split(fireX, firey, test_size=0.3, random_state=1, stratify = firey)
Xtr3, Xte3, ytr3, yte3 = train_test_split(partyX, partyy, test_size=0.3, random_state=1, stratify = partyy)

Now that we have training and testing data for each of our collection of tweets, I'm going to be looking at the baseline performances of four models on each of our data sets. The four models are the following:

- Logistic Regression
- Random Forest Classifier
- XGBClassifier
- SVC

I'm going to determine the performance of each of these models using a baseline version of each. Each different type of model will be tested on each of our sets of data, which means we are going to end up with 12 sets of results in the end, that will be compiled into a table. 

Below defined are helper functions that will help not only to create baseline models but to determine their performance by printing out classification reports and confusion matrices. 

In [145]:
from matplotlib.ticker import IndexLocator
import itertools
from sklearn.metrics import classification_report

def plot_cm(y_test,y_pred_class,classes=['NON-default','DEFAULT']):
    # plot confusion matrix
    fig, ax = plt.subplots()
    cm = confusion_matrix(y_test, y_pred_class)
    
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    plt.title("Confusion Matrix")
    ax.set(yticks=[-0.5, 1.5], 
           xticks=[0, 1], 
           yticklabels=classes, 
           xticklabels=classes)
    ax.yaxis.set_major_locator(IndexLocator(base=1, offset=0.5))
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
def scores(y_test, y_pred_class):
    # Prints formatted classification metrics. 
    #print('Classification Accuracy: ', format(accuracy_score(y_test, y_pred_class), '.3f'))
    #print('Precision score: ', format(precision_score(y_test, y_pred_class), '.3f'))
    #print('Recall score: ', format(recall_score(y_test, y_pred_class), '.3f'))
    #print('F1 score: ', format(f1_score(y_test, y_pred_class), '.3f'))
    
    target_names = ['positive tweets', 'negative tweets']
    print(classification_report(y_test, y_pred_class, target_names=target_names))

In [146]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.svm import SVC

def logiRegr(X_train, y_train, X_test, y_test,**kwargs):
    # Instantiate model. Use kwargs to pass parameters.
    # Pass GridSearch best_params with ** to unpack.
    logreg = LogisticRegression(random_state=1,**kwargs)
    # Fit to training data.
    logreg.fit(X_train, y_train)
    # Examine coefficients
    #pprint.pprint(list(zip(X_train.columns,logreg.coef_[0])))
    # Class predictions (not predicted probabilities)
    print("Performance on Training Data")
    y_pred_class = logreg.predict(X_train)
    scores(y_train, y_pred_class)
    
    print("Performance on Testing Data")
    y_pred_class = logreg.predict(X_test)
    scores(y_test, y_pred_class)
    # Plot confusion matrix
    #plot_cm(y_test,y_pred_class)
    
def randomForest(X_train, y_train, X_test, y_test,**kwargs):
    # Instantiate model. Use kwargs to pass parameters.
    # Pass GridSearch best_params with ** to unpack.
    rf = RandomForestClassifier(random_state=1, **kwargs) 
    # Fit to training data.
    rf.fit(X_train,y_train)
    # Class predictions
    y_pred_class = rf.predict(X_test)
    # Scoring metrics
    scores(y_test, y_pred_class)
    # Confusion matrix
    #plot_cm(y_test,y_pred_class)
    
def xgbClass(X_train, y_train, X_test, y_test,**kwargs):
    # Instantiate model. Use kwargs to pass parameters.
    # Pass GridSearch best_params with ** to unpack.
    xg = xgb.XGBClassifier(seed=1,**kwargs)
    # Fit to training data.
    xg.fit(X_train,y_train)
    # Class predictions
    y_pred_class = xg.predict(X_test)
    # Scoring metrics
    scores(y_test, y_pred_class)
    # Confusion matrix
    #plot_cm(y_test,y_pred_class)
    
def svmClass(X_train, y_train, X_test, y_test, **kwargs):
    # Instantiate model. Use kwargs to pass parameters.
    # Pass GridSearch best_params with ** to unpack.
    svm = SVC(random_state=1,**kwargs)
    # Fit to training data.
    svm.fit(X_train, y_train)
    # Class predictions
    print("Performance on Training Data")
    y_pred_class = svm.predict(X_train)
    scores(y_train, y_pred_class)
    
    print("Performance on Testing Data")
    y_pred_class = svm.predict(X_test)
    scores(y_test, y_pred_class)

Here we're going to be creating and evaluating baseline models for each game. 4 types of models for 3 games, feel free to skip the lengthy amount of output, there will be a table at the end that summarizes the results.

In [147]:
logiRegr(Xtr1, ytr1, Xte1, yte1)

Performance on Training Data
                 precision    recall  f1-score   support

positive tweets       0.99      1.00      1.00      8052
negative tweets       0.98      0.85      0.91       458

       accuracy                           0.99      8510
      macro avg       0.98      0.92      0.95      8510
   weighted avg       0.99      0.99      0.99      8510

Performance on Testing Data
                 precision    recall  f1-score   support

positive tweets       0.98      1.00      0.99      3452
negative tweets       0.90      0.70      0.79       196

       accuracy                           0.98      3648
      macro avg       0.94      0.85      0.89      3648
   weighted avg       0.98      0.98      0.98      3648



In [148]:
randomForest(Xtr1, ytr1, Xte1, yte1)

                 precision    recall  f1-score   support

positive tweets       0.99      0.99      0.99      3452
negative tweets       0.88      0.82      0.85       196

       accuracy                           0.98      3648
      macro avg       0.93      0.90      0.92      3648
   weighted avg       0.98      0.98      0.98      3648



In [149]:
xgbClass(Xtr1, ytr1, Xte1, yte1)

                 precision    recall  f1-score   support

positive tweets       0.99      0.99      0.99      3452
negative tweets       0.89      0.77      0.83       196

       accuracy                           0.98      3648
      macro avg       0.94      0.88      0.91      3648
   weighted avg       0.98      0.98      0.98      3648



In [150]:
svmClass(Xtr1, ytr1, Xte1, yte1)

Performance on Training Data
                 precision    recall  f1-score   support

positive tweets       0.99      1.00      1.00      8052
negative tweets       0.98      0.89      0.93       458

       accuracy                           0.99      8510
      macro avg       0.99      0.94      0.96      8510
   weighted avg       0.99      0.99      0.99      8510

Performance on Testing Data
                 precision    recall  f1-score   support

positive tweets       0.98      1.00      0.99      3452
negative tweets       0.94      0.70      0.80       196

       accuracy                           0.98      3648
      macro avg       0.96      0.85      0.90      3648
   weighted avg       0.98      0.98      0.98      3648



In [151]:
logiRegr(Xtr2, ytr2, Xte2, yte2)

Performance on Training Data
                 precision    recall  f1-score   support

positive tweets       0.99      1.00      1.00       912
negative tweets       1.00      0.65      0.79        17

       accuracy                           0.99       929
      macro avg       1.00      0.82      0.89       929
   weighted avg       0.99      0.99      0.99       929

Performance on Testing Data
                 precision    recall  f1-score   support

positive tweets       0.99      0.99      0.99       391
negative tweets       0.67      0.50      0.57         8

       accuracy                           0.98       399
      macro avg       0.83      0.75      0.78       399
   weighted avg       0.98      0.98      0.98       399



In [152]:
randomForest(Xtr2, ytr2, Xte2, yte2)

                 precision    recall  f1-score   support

positive tweets       0.98      1.00      0.99       391
negative tweets       1.00      0.25      0.40         8

       accuracy                           0.98       399
      macro avg       0.99      0.62      0.70       399
   weighted avg       0.99      0.98      0.98       399



In [153]:
xgbClass(Xtr2, ytr2, Xte2, yte2)

                 precision    recall  f1-score   support

positive tweets       0.99      0.99      0.99       391
negative tweets       0.67      0.50      0.57         8

       accuracy                           0.98       399
      macro avg       0.83      0.75      0.78       399
   weighted avg       0.98      0.98      0.98       399



In [154]:
svmClass(Xtr2, ytr2, Xte2, yte2)

Performance on Training Data
                 precision    recall  f1-score   support

positive tweets       1.00      1.00      1.00       912
negative tweets       1.00      0.76      0.87        17

       accuracy                           1.00       929
      macro avg       1.00      0.88      0.93       929
   weighted avg       1.00      1.00      1.00       929

Performance on Testing Data
                 precision    recall  f1-score   support

positive tweets       0.98      1.00      0.99       391
negative tweets       1.00      0.25      0.40         8

       accuracy                           0.98       399
      macro avg       0.99      0.62      0.70       399
   weighted avg       0.99      0.98      0.98       399



In [155]:
logiRegr(Xtr3, ytr3, Xte3, yte3)

Performance on Training Data
                 precision    recall  f1-score   support

positive tweets       0.99      1.00      0.99       493
negative tweets       1.00      0.77      0.87        31

       accuracy                           0.99       524
      macro avg       0.99      0.89      0.93       524
   weighted avg       0.99      0.99      0.99       524

Performance on Testing Data
                 precision    recall  f1-score   support

positive tweets       0.95      1.00      0.97       211
negative tweets       1.00      0.21      0.35        14

       accuracy                           0.95       225
      macro avg       0.98      0.61      0.66       225
   weighted avg       0.95      0.95      0.94       225



In [156]:
randomForest(Xtr3, ytr3, Xte3, yte3)

                 precision    recall  f1-score   support

positive tweets       0.95      1.00      0.97       211
negative tweets       0.67      0.14      0.24        14

       accuracy                           0.94       225
      macro avg       0.81      0.57      0.60       225
   weighted avg       0.93      0.94      0.92       225



In [157]:
xgbClass(Xtr3, ytr3, Xte3, yte3)

                 precision    recall  f1-score   support

positive tweets       0.95      0.98      0.97       211
negative tweets       0.50      0.29      0.36        14

       accuracy                           0.94       225
      macro avg       0.73      0.63      0.67       225
   weighted avg       0.93      0.94      0.93       225



In [158]:
svmClass(Xtr3, ytr3, Xte3, yte3)

Performance on Training Data
                 precision    recall  f1-score   support

positive tweets       0.99      1.00      0.99       493
negative tweets       1.00      0.77      0.87        31

       accuracy                           0.99       524
      macro avg       0.99      0.89      0.93       524
   weighted avg       0.99      0.99      0.99       524

Performance on Testing Data
                 precision    recall  f1-score   support

positive tweets       0.94      1.00      0.97       211
negative tweets       1.00      0.07      0.13        14

       accuracy                           0.94       225
      macro avg       0.97      0.54      0.55       225
   weighted avg       0.95      0.94      0.92       225



There are disparities in the performance of our Logisitic Regression and SVM models when they are predicting training and testing data. This indicates overfitting, and additional hyperparameter tuning will be performed to address this.

Due to the imbalanced nature of our data set, the challenge here is building a model that can more accurately identify the minority class, which in this case is 'negative tweets'. Because the models will have little issue identifying positive tweets either way, the key metric of performance here will be the recall score on negative tweets.

To summarize the proportion of positive and negative labels in our training data sets:

In [23]:
ytr1.value_counts()

0    8052
1     458
Name: label, dtype: int64

In [24]:
ytr2.value_counts()

0    912
1     17
Name: label, dtype: int64

In [25]:
ytr3.value_counts()

0    493
1     31
Name: label, dtype: int64

Based on this information, we can assign weights to each class of each training data set, to use in tuning our models.

In [40]:
weights1 = {0: 1, 1: 8052/458}
weights2 = {0: 1, 1: 912/17}
weights3 = {0: 1, 1: 493/31}

# Summary of Recall Scores of Negative Tweets

Game1: Smash Bros Ultimate

Game2: Fire Emblem: Three Houses

Game3: Super Mario Party


|      |Game1 |Game2 |Game3 |
|------|------|------|------|
|LogReg|0.700 |0.500 |0.210 |
|RanFor|0.820 |0.250 |0.140 |
|XGB   |0.770 |0.500 |0.290 |
|SVM   |0.700 |0.250 |0.070 |

# Hyperparameter Tuning

In [46]:
# Function that prints results of gridsearch of hyperparameters

def printGridResult(grid_result):
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

WARNING: Long walls of outputs incoming. 12 sets of hyperparameter tuning performed, one for each model for each game (4x3). Results summarized at the end.

Note: many different variable names are defined and deleted after they're done being used to ensure that as much memory is freed up as possible, in case python's garbage collection isn't doing a good enough job.

# Logistic Regression Models Hyperparameter Tuning

The baseline LogisticRegression model has default parameters of solver='lbgfs' and C=1.0. Let's test some different values for these parameters to see if there's a model with different parameters that yields a higher recall score.

In [48]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

solvers = ['newton-cg', 'lbfgs', 'saga']
c_values = [100, 10, 1.0, 0.1, 0.01]
class_weight = [None, weights1]

grid = dict(solver=solvers,C=c_values, class_weight=class_weight)
model1 = LogisticRegression()

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
grid_search1 = GridSearchCV(estimator=model1, param_grid=grid, n_jobs=4, cv=cv, scoring='recall',error_score=0)
grid_result1 = grid_search1.fit(Xtr1, ytr1)
printGridResult(grid_result1)
del grid_result1, grid_search1, model1

Best: 0.932370 using {'C': 0.01, 'class_weight': {0: 1, 1: 17.580786026200872}, 'solver': 'saga'}


In [49]:
class_weight = [None, weights2]

grid = dict(solver=solvers,C=c_values, class_weight=class_weight)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)

model2 = LogisticRegression()
grid_search2 = GridSearchCV(estimator=model2, param_grid=grid, n_jobs=4, cv=cv, scoring='recall',error_score=0)
grid_result2 = grid_search2.fit(Xtr2, ytr2)
printGridResult(grid_result2)
del grid_search2, grid_result2, model2

Best: 1.000000 using {'C': 0.01, 'class_weight': {0: 1, 1: 53.64705882352941}, 'solver': 'newton-cg'}


In [50]:
class_weight = [None, weights3]

grid = dict(solver=solvers,C=c_values, class_weight=class_weight)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)

model3 = LogisticRegression()
grid_search3 = GridSearchCV(estimator=model3, param_grid=grid, n_jobs=4, cv=cv, scoring='recall',error_score=0)
grid_result3 = grid_search3.fit(Xtr3, ytr3)
printGridResult(grid_result3)
del grid_search3, grid_result3, model3

Best: 0.904762 using {'C': 0.01, 'class_weight': {0: 1, 1: 15.903225806451612}, 'solver': 'newton-cg'}


# Random Forest Models Hyperparameter Tuning

The baseline RandomForestClassifier has a default of n_estimators=100. More trees in general will always improve the performance of the model, so we'll test n_estimators=1000 to see if there's a significant improvement in performance. We'll also play around with the max_features parameter. 

In [51]:
model4 = RandomForestClassifier()

class_weight = [None, weights1]
n_estimators = [100, 1000]
max_features = ['sqrt', 'log2']
grid = dict(class_weight=class_weight, n_estimators=n_estimators, max_features=max_features)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)

grid_search4 = GridSearchCV(estimator=model4, param_grid=grid, n_jobs=4, cv=cv, scoring='recall',error_score=0)
grid_result4 = grid_search4.fit(Xtr1, ytr1)
printGridResult(grid_result4)
del grid_search4, grid_result4, model4

Best: 0.827520 using {'class_weight': None, 'max_features': 'sqrt', 'n_estimators': 100}


In [52]:
model5 = RandomForestClassifier()

class_weight = [None, weights2]
grid = dict(class_weight=class_weight, n_estimators=n_estimators, max_features=max_features)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)

grid_search5 = GridSearchCV(estimator=model5, param_grid=grid, n_jobs=4, cv=cv, scoring='recall',error_score=0)
grid_result5 = grid_search5.fit(Xtr2, ytr2)
printGridResult(grid_result5)
del grid_search5, grid_result5, model5

Best: 0.116667 using {'class_weight': None, 'max_features': 'sqrt', 'n_estimators': 1000}


In [53]:
model6 = RandomForestClassifier()

class_weight = [None, weights3]
grid = dict(class_weight=class_weight, n_estimators=n_estimators, max_features=max_features)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)

grid_search6 = GridSearchCV(estimator=model6, param_grid=grid, n_jobs=4, cv=cv, scoring='recall',error_score=0)
grid_result6 = grid_search6.fit(Xtr3, ytr3)
printGridResult(grid_result6)
del grid_search6, grid_result6, model6

Best: 0.280952 using {'class_weight': None, 'max_features': 'sqrt', 'n_estimators': 100}


# XGBoost Models Hyperparameter Tuning

The default parameters for XGBClassifier are max_depth=3, n_estimators=100, and learning_rate=0.1. We'll slightly increase max_depth while testing for different values of the other parameters.

In [54]:
model7 = xgb.XGBClassifier(max_depth=5)

scale_pos_weight = [1, 8052/458]
learning_rate = [0.001, 0.01, 0.1]
n_estimators = [100, 1000]

grid = dict(scale_pos_weight=scale_pos_weight, learning_rate=learning_rate, n_estimators=n_estimators)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)

grid_search7 = GridSearchCV(estimator=model7, param_grid=grid, n_jobs=4, cv=cv, scoring='recall',error_score=0)
grid_result7 = grid_search7.fit(Xtr1, ytr1)
printGridResult(grid_result7)
del grid_search7, grid_result7, model7

Best: 0.984783 using {'learning_rate': 0.001, 'n_estimators': 1000, 'scale_pos_weight': 17.580786026200872}


In [55]:
model8 = xgb.XGBClassifier(max_depth=5)

scale_pos_weight = [1, 912/17]
grid = dict(scale_pos_weight=scale_pos_weight, learning_rate=learning_rate, n_estimators=n_estimators)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)

grid_search8 = GridSearchCV(estimator=model8, param_grid=grid, n_jobs=4, cv=cv, scoring='recall',error_score=0)
grid_result8 = grid_search8.fit(Xtr2, ytr2)
printGridResult(grid_result8)
del grid_search8, grid_result8, model8

Best: 0.766667 using {'learning_rate': 0.001, 'n_estimators': 100, 'scale_pos_weight': 53.64705882352941}


In [56]:
model9 = xgb.XGBClassifier(max_depth=5)

scale_pos_weight = [1, 493/31]
grid = dict(scale_pos_weight=scale_pos_weight, learning_rate=learning_rate, n_estimators=n_estimators)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)

grid_search9 = GridSearchCV(estimator=model9, param_grid=grid, n_jobs=4, cv=cv, scoring='recall',error_score=0)
grid_result9 = grid_search9.fit(Xtr3, ytr3)
printGridResult(grid_result9)
del grid_search9, grid_result9, model9

Best: 0.904762 using {'learning_rate': 0.001, 'n_estimators': 100, 'scale_pos_weight': 15.903225806451612}


# SVM Models Hyperparameter Tuning

The baseline SVC model's default parameters are kernel='rbf', C=1.0. We'll play around with different values of these parameters.

In [57]:
model10 = SVC()

class_weight = [None, weights1]
kernel = ['poly', 'rbf', 'sigmoid']
C = [50, 10, 1.0, 0.1, 0.01]

# define grid search
grid = dict(class_weight=class_weight, kernel=kernel,C=C)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)

grid_search10 = GridSearchCV(estimator=model10, param_grid=grid, n_jobs=4, cv=cv, scoring='recall',error_score=0)
grid_result10 = grid_search10.fit(Xtr1, ytr1)
printGridResult(grid_result10)
del grid_search10, grid_result10, model10

/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best: 0.997826 using {'C': 0.01, 'class_weight': {0: 1, 1: 17.580786026200872}, 'kernel': 'poly'}


In [58]:
model11 = SVC()

class_weight = [None, weights2]

# define grid search
grid = dict(class_weight=class_weight, kernel=kernel,C=C)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
grid_search11 = GridSearchCV(estimator=model11, param_grid=grid, n_jobs=4, cv=cv, scoring='recall',error_score=0)
grid_result11 = grid_search11.fit(Xtr2, ytr2)
printGridResult(grid_result11)
del grid_search11, grid_result11, model11

Best: 1.000000 using {'C': 0.01, 'class_weight': {0: 1, 1: 53.64705882352941}, 'kernel': 'poly'}


In [59]:
model12 = SVC()

class_weight = [None, weights3]

grid = dict(class_weight=class_weight, kernel=kernel,C=C)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
grid_search12 = GridSearchCV(estimator=model12, param_grid=grid, n_jobs=4, cv=cv, scoring='recall',error_score=0)
grid_result12 = grid_search12.fit(Xtr3, ytr3)
printGridResult(grid_result12)
del grid_search12, grid_result12, model12

Best: 1.000000 using {'C': 0.01, 'class_weight': {0: 1, 1: 15.903225806451612}, 'kernel': 'poly'}


Taking the best hyperparameters for each model per game, our recall scores are now the following:


|      |Game1 |Game2 |Game3 |
|------|------|------|------|
|LogReg|0.932 |1.000 |0.905 |
|RanFor|0.828 |0.117 |0.281 |
|XGB   |0.985 |0.767 |0.905 |
|SVM   |0.998 |1.000 |1.000 |

Because during hyperparameter tuning, we only looked at overall recall instead of recall of only negative tweets, the results look much better than they actually are. But based off of this, we can at least understand which models perform better relatively.

Based on the hyperparameter tuning, we'll build models according to the best hyperparameters and fine tune the class weights further until the models' classification performance lands in a more desirable place. 

- Game 1 (Super Smash Bros Ultimate): SVM
- Game 2 (Fire Emblem: Three Houses): LogReg, SVM
- Game 3 (Super Mario Party): SVM


Let's build each of these models with the optimal hyperparameters, starting with Smash Bros.

# Extended Model Tuning - Class Weights

In [159]:
from sklearn.model_selection import cross_validate

smashmodel = SVC(C=0.01, class_weight=weights1, kernel='poly')
smashmodel.fit(Xtr1, ytr1)

ypred = smashmodel.predict(Xte1)
scores(yte1, ypred)


                 precision    recall  f1-score   support

positive tweets       1.00      0.63      0.77      3452
negative tweets       0.13      1.00      0.23       196

       accuracy                           0.65      3648
      macro avg       0.57      0.81      0.50      3648
   weighted avg       0.95      0.65      0.74      3648



The model seems to be overly sensitive about labeling tweets as negative now. A recall of 1.00 for the negative tweets suggests that while the model was able to catch every negatively labeled tweet as negative, it did so by aggressively predicting "negative" at the expense of mislabeling many of the positive tweets. This is further supported by the low precision score when it comes to negative tweets. Let's perform some further adjusting on the weights to see if we can find a reasonable compromise.

The weights are too "heavy" if they're causing our models to become too paranoid over mislabeling negative tweets. Therefore we need to lower the weights until a reasonable balance between its performance when it comes to classifying positive and negative tweets.

After manual trial and error, the model performance was found to be at its most balanced when the negative tweet class weight was reduced by a factor of 1.37. 

In [160]:
adjust = 1.37
weightstest = {0:1, 1: (8052 / (452 * adjust))}

smashmodel = SVC(C=0.01, class_weight=weightstest, kernel='poly')
smashmodel.fit(Xtr1, ytr1)

ypred = smashmodel.predict(Xte1)
scores(yte1, ypred)

                 precision    recall  f1-score   support

positive tweets       0.99      0.94      0.97      3452
negative tweets       0.45      0.83      0.59       196

       accuracy                           0.94      3648
      macro avg       0.72      0.89      0.78      3648
   weighted avg       0.96      0.94      0.95      3648



Looking much better, recall for negative tweets while slightly lower is far more precise, and is no longer interfering with classification of positive tweets. On a more general note, we can also see that the overall accuracy is much higher, starting from 0.65 and ending at 0.94.

Moving on to Fire Emblem's best performing model:

In [161]:
# Fire Emblem's best performing model, with hyperparameters tuned

firemodel = LogisticRegression(C=0.01, class_weight=weights2, solver='newton-cg')
firemodel.fit(Xtr2, ytr2)

ypred = firemodel.predict(Xte2)
scores(yte2, ypred)

                 precision    recall  f1-score   support

positive tweets       1.00      0.85      0.92       391
negative tweets       0.12      1.00      0.21         8

       accuracy                           0.85       399
      macro avg       0.56      0.92      0.57       399
   weighted avg       0.98      0.85      0.90       399



Again with the same issue of the weights being too skewed towards labeling the negative tweets. High recall with low precision means that the model is just guessing negative more often just to be safe. While we don't mind more false positives, let's see if we can adjust the weights more to improve the precision of negative tweet predictions.

In [94]:
adjust = 2.6
weightstest = {0:1, 1: (912 / (17 * adjust))}

firemodel = LogisticRegression(C=0.01, class_weight=weightstest, solver='newton-cg')
firemodel.fit(Xtr2, ytr2)

ypred = firemodel.predict(Xte2)
scores(yte2, ypred)

                 precision    recall  f1-score   support

positive tweets       1.00      0.98      0.99       391
 negativetweets       0.47      0.88      0.61         8

       accuracy                           0.98       399
      macro avg       0.73      0.93      0.80       399
   weighted avg       0.99      0.98      0.98       399



Much more preferable, improved scores across the board, with only a very slight hit to the recall of negative tweets.

In [95]:
firemodel = SVC(C=0.01, class_weight=weights2, kernel='poly')
firemodel.fit(Xtr2, ytr2)

ypred = firemodel.predict(Xte2)
scores(yte2, ypred)

                 precision    recall  f1-score   support

positive tweets       1.00      0.82      0.90       391
 negativetweets       0.10      1.00      0.19         8

       accuracy                           0.82       399
      macro avg       0.55      0.91      0.54       399
   weighted avg       0.98      0.82      0.89       399



In [102]:
weightstest = {0:1, 1: (912 / (17 * 2.6))}

firemodel = SVC(C=0.01, class_weight=weightstest, kernel='poly')
firemodel.fit(Xtr2, ytr2)

ypred = firemodel.predict(Xte2)
scores(yte2, ypred)

                 precision    recall  f1-score   support

positive tweets       0.99      0.99      0.99       391
 negativetweets       0.71      0.62      0.67         8

       accuracy                           0.99       399
      macro avg       0.85      0.81      0.83       399
   weighted avg       0.99      0.99      0.99       399



In [132]:
firemodel = SVC(C=0.01, class_weight=weightstest, kernel='poly')
firemodel.fit(Xtr2, ytr2)

SVC(C=0.01, break_ties=False, cache_size=200,
    class_weight={0: 1, 1: 11.45352956892446}, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [103]:
# Mario Party's best performing model

partymodel = SVC(C=0.01, class_weight=weights3, kernel='poly')
partymodel.fit(Xtr3, ytr3)

ypred = partymodel.predict(Xte3)
scores(yte3, ypred)

                 precision    recall  f1-score   support

positive tweets       1.00      0.35      0.51       211
 negativetweets       0.09      1.00      0.17        14

       accuracy                           0.39       225
      macro avg       0.55      0.67      0.34       225
   weighted avg       0.94      0.39      0.49       225



In [131]:
weightstest = {0: 1, 1: 493 / (31 * 1.3885)}

partymodel = SVC(C=0.01, class_weight=weightstest, kernel='poly')
partymodel.fit(Xtr3, ytr3)

ypred = partymodel.predict(Xte3)
scores(yte3, ypred)

                 precision    recall  f1-score   support

positive tweets       0.98      0.95      0.96       211
 negativetweets       0.45      0.64      0.53        14

       accuracy                           0.93       225
      macro avg       0.71      0.80      0.75       225
   weighted avg       0.94      0.93      0.93       225



# Tree Model Tuning - Class Weights

In [166]:
# XGBClassifier for Smash Bros

smashXGB = xgb.XGBClassifier(max_depth=5, learning_rate=0.001, n_estimators=1000, scale_pos_weight=(8052/458))
smashXGB.fit(Xtr1, ytr1)
ypred = smashXGB.predict(Xte1)

scores(yte1, ypred)

                 precision    recall  f1-score   support

positive tweets       1.00      0.68      0.81      3452
negative tweets       0.15      0.98      0.26       196

       accuracy                           0.69      3648
      macro avg       0.57      0.83      0.53      3648
   weighted avg       0.95      0.69      0.78      3648



In [167]:
# XGBClassifier for Smash Bros

adjust=1.5

smashXGB = xgb.XGBClassifier(max_depth=5, learning_rate=0.001, n_estimators=1000, scale_pos_weight=(8052/(458*adjust)))
smashXGB.fit(Xtr1, ytr1)
ypred = smashXGB.predict(Xte1)

scores(yte1, ypred)

                 precision    recall  f1-score   support

positive tweets       0.98      0.99      0.99      3452
negative tweets       0.79      0.71      0.75       196

       accuracy                           0.97      3648
      macro avg       0.89      0.85      0.87      3648
   weighted avg       0.97      0.97      0.97      3648



In [185]:
# XGBClassifier for Smash Bros

fireXGB = xgb.XGBClassifier(max_depth=5, learning_rate=0.001, n_estimators=100, scale_pos_weight=(912/17))
fireXGB.fit(Xtr2, ytr2)
ypred = fireXGB.predict(Xte2)

scores(yte2, ypred)

                 precision    recall  f1-score   support

positive tweets       1.00      0.90      0.95       391
negative tweets       0.17      1.00      0.29         8

       accuracy                           0.90       399
      macro avg       0.58      0.95      0.62       399
   weighted avg       0.98      0.90      0.93       399



In [183]:
# XGBClassifier for Smash Bros

adjust=10

fireXGB = xgb.XGBClassifier(max_depth=5, learning_rate=0.001, n_estimators=100, scale_pos_weight=(912/(17*adjust)))
fireXGB.fit(Xtr2, ytr2)
ypred = fireXGB.predict(Xte2)

scores(yte2, ypred)

                 precision    recall  f1-score   support

positive tweets       0.99      0.99      0.99       391
negative tweets       0.67      0.50      0.57         8

       accuracy                           0.98       399
      macro avg       0.83      0.75      0.78       399
   weighted avg       0.98      0.98      0.98       399



In [186]:
# XGBClassifier for Smash Bros

partyXGB = xgb.XGBClassifier(max_depth=5, learning_rate=0.001, n_estimators=100, scale_pos_weight=(493/31))
partyXGB.fit(Xtr3, ytr3)
ypred = partyXGB.predict(Xte3)

scores(yte3, ypred)

                 precision    recall  f1-score   support

positive tweets       0.98      0.77      0.86       211
negative tweets       0.17      0.71      0.28        14

       accuracy                           0.77       225
      macro avg       0.57      0.74      0.57       225
   weighted avg       0.93      0.77      0.83       225



In [192]:
# XGBClassifier for Smash Bros

adjust=6

partyXGB = xgb.XGBClassifier(max_depth=5, learning_rate=0.001, n_estimators=100, scale_pos_weight=(493/(31*adjust)))
partyXGB.fit(Xtr3, ytr3)
ypred = partyXGB.predict(Xte3)

scores(yte3, ypred)

                 precision    recall  f1-score   support

positive tweets       0.97      0.99      0.98       211
negative tweets       0.73      0.57      0.64        14

       accuracy                           0.96       225
      macro avg       0.85      0.78      0.81       225
   weighted avg       0.96      0.96      0.96       225



# Parametric Model Tuning - Class Weights

In [194]:
# Fire Emblem's best performing model, with hyperparameters tuned

smashLR = LogisticRegression(C=0.01, class_weight=weights1, solver='saga')
smashLR.fit(Xtr1, ytr1)

ypred = smashLR.predict(Xte1)
scores(yte1, ypred)

                 precision    recall  f1-score   support

positive tweets       0.99      0.90      0.94      3452
negative tweets       0.34      0.91      0.49       196

       accuracy                           0.90      3648
      macro avg       0.67      0.90      0.72      3648
   weighted avg       0.96      0.90      0.92      3648



In [211]:
# Fire Emblem's best performing model, with hyperparameters tuned

adjust=1.5
weightstest = {0:1, 1: (8052 / (452 * adjust))}

smashLR = LogisticRegression(C=0.01, class_weight=weightstest, solver='saga')
smashLR.fit(Xtr1, ytr1)

ypred = smashLR.predict(Xte1)
scores(yte1, ypred)

                 precision    recall  f1-score   support

positive tweets       0.99      0.96      0.97      3452
negative tweets       0.52      0.83      0.64       196

       accuracy                           0.95      3648
      macro avg       0.76      0.89      0.81      3648
   weighted avg       0.96      0.95      0.96      3648



In [212]:
# Fire Emblem's best performing model, with hyperparameters tuned

fireLR = LogisticRegression(C=0.01, class_weight=weights2, solver='newton-cg')
fireLR.fit(Xtr2, ytr2)

ypred = fireLR.predict(Xte2)
scores(yte2, ypred)

                 precision    recall  f1-score   support

positive tweets       1.00      0.85      0.92       391
negative tweets       0.12      1.00      0.21         8

       accuracy                           0.85       399
      macro avg       0.56      0.92      0.57       399
   weighted avg       0.98      0.85      0.90       399



In [238]:
adjust=2.69
weightstest = {0:1, 1: (912 / (17 * adjust))}

fireLR = LogisticRegression(C=0.01, class_weight=weightstest, solver='newton-cg')
fireLR.fit(Xtr2, ytr2)

ypred = fireLR.predict(Xte2)
scores(yte2, ypred)

                 precision    recall  f1-score   support

positive tweets       1.00      0.98      0.99       391
negative tweets       0.50      0.88      0.64         8

       accuracy                           0.98       399
      macro avg       0.75      0.93      0.81       399
   weighted avg       0.99      0.98      0.98       399



In [239]:
# Fire Emblem's best performing model, with hyperparameters tuned

partyLR = LogisticRegression(C=0.01, class_weight=weights3, solver='newton-cg')
partyLR.fit(Xtr3, ytr3)

ypred = partyLR.predict(Xte3)
scores(yte3, ypred)

                 precision    recall  f1-score   support

positive tweets       1.00      0.69      0.81       211
negative tweets       0.17      1.00      0.30        14

       accuracy                           0.71       225
      macro avg       0.59      0.84      0.56       225
   weighted avg       0.95      0.71      0.78       225



In [254]:
adjust=1.8
weightstest = {0:1, 1: (493 / (31 * adjust))}

partyLR = LogisticRegression(C=0.01, class_weight=weightstest, solver='newton-cg')
partyLR.fit(Xtr3, ytr3)

ypred = partyLR.predict(Xte3)
scores(yte3, ypred)

                 precision    recall  f1-score   support

positive tweets       0.98      0.96      0.97       211
negative tweets       0.53      0.64      0.58        14

       accuracy                           0.94       225
      macro avg       0.75      0.80      0.77       225
   weighted avg       0.95      0.94      0.94       225



# Summary of Adjusted Weights

|      |Game1 |Game2 |Game3 |
|------|------|------|------|
|LogReg|1.5   |2.69  |1.8   |
|XGB   |1.5   |10    |6     |
|SVM   |1.37  |2.6   |1.3885|

# Evaluation Against Manually Labeled Data

Going into this, I expect the results to be quite catastrophic actually. While labeling sample data myself, I noticed that there were very rarely tweets that conveyed clear negative sentiment. What doesn't help is when tweets often use a lot of negative words but to actually convey positive sentiment. For example, a tweet in response to Mario Party says, "Oh God no, not Mario Party! That game destroys friendships I tell ya!". Even a human might not initially perceive this tweet as negative without understanding the context of the game. Mario Party is a party game where players need to get themselves ahead while also sabotaging other players to keep them behind. Hence, the "destroys friendships" part. 

Let's just see what the results are.

In [256]:
# Evaluating Smash Bros model against manually labeled tweets

adjust = 1.37
weightstest = {0:1, 1: (8052 / (452 * adjust))}

smashmodel = SVC(C=0.01, class_weight=weightstest, kernel='poly')
smashmodel.fit(Xtr1, ytr1)

smashsamplevec = smashvectorizer.transform(smashsamples['cleanedtext']).toarray()
smashsamplepred = smashmodel.predict(smashsamplevec)
scores(smashsamplepred, smashsamples['label'])

                 precision    recall  f1-score   support

positive tweets       0.92      0.99      0.95       342
negative tweets       0.50      0.11      0.19        35

       accuracy                           0.91       377
      macro avg       0.71      0.55      0.57       377
   weighted avg       0.88      0.91      0.88       377



In [257]:
# Evaluating Fire Emblem model against manually labeled tweets

adjust = 2.6
weightstest = {0:1, 1: (912 / (17 * adjust))}

firemodel = LogisticRegression(C=0.01, class_weight=weightstest, solver='newton-cg')
firemodel.fit(Xtr2, ytr2)

firesamplevec = firevectorizer.transform(firesamples['cleanedtext']).toarray()
firesamplepred = firemodel.predict(firesamplevec)
scores(firesamplepred, firesamples['label'])

                 precision    recall  f1-score   support

positive tweets       0.96      0.98      0.97       226
negative tweets       0.00      0.00      0.00         9

       accuracy                           0.94       235
      macro avg       0.48      0.49      0.48       235
   weighted avg       0.92      0.94      0.93       235



In [258]:
# Evaluating Mario Party model against manually labeled tweets

weightstest = {0: 1, 1: 493 / (31 * 1.3885)}

partymodel = SVC(C=0.01, class_weight=weightstest, kernel='poly')
partymodel.fit(Xtr3, ytr3)

partysamplevec = partyvectorizer.transform(partysamples['cleanedtext']).toarray()
partysamplepred = partymodel.predict(partysamplevec)
scores(partysamplepred, partysamples['label'])

                 precision    recall  f1-score   support

positive tweets       0.94      0.98      0.96       124
negative tweets       0.25      0.11      0.15         9

       accuracy                           0.92       133
      macro avg       0.59      0.54      0.56       133
   weighted avg       0.89      0.92      0.90       133



# Discussion of Results

These results are rather damning, but not all that suprising. Our data is inherently very imbalanced after being labeled by TextBlob, and even more imbalanced when manually labeled. Furthermore, our approach of representing tweets using bag-of-words representations is likely too crude and loses too much information such as phrases and sentence structure. We will continue on with further experimentation with the modeling and the insights that we can draw from them.

# Conclusions

Unfortunately our modeling turned out to be a failure. Due to the nature of our data, it was very important for our training data and model to be complex enough to be able to accurately detect the rare negative tweets. In other words the tolerance for our false postive rate was extremely low, for the models to be considered acceptable. The problem is that our bag-of-word representations lack the nuance to truly capture sentiment of the original tweets. Not only this, but TextBlob is most likely not an accurate labeler of our documents (tweets) here either. TextBlob's sentiment analysis is based on a model trained on movie reviews. The language used in movie reviews include many more clear postive or negative words, whereas the language used in tweets are more unfocused and include much more slang. Therefore even a bag-of-word representation of a movie review would still retain enough information to reasonably evaluate sentiment off of. Moving forward, the first step to creating models that accurately label tweets, could be to have a large enough training data set of accurately labeled tweets. 